# Import relevant libraries

In [2]:
import seaborn as sns
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import math

# Interpreting Logicstic Regression

**Lets use the Titanic dataset**

In [3]:
titanic = sns.load_dataset("titanic")
titanic.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


In [4]:
model1 = smf.logit(formula='survived ~ 1', data=titanic).fit();
model1.params

Optimization terminated successfully.
         Current function value: 0.665912
         Iterations 4


Intercept   -0.473288
dtype: float64

**How does this translate into probability?**

In [5]:
log_odd=  -0.47
odds = math.exp(log_odd)
print("Odds:", odds)
probability = odds/(1+odds)
print("Probability:", probability)

Odds: 0.6250022682827008
Probability: 0.3846162436088178


**Let's double check this**

In [6]:
cross_tab = pd.DataFrame({
    'count': titanic['survived'].value_counts(), 
    'percentage': titanic['survived'].value_counts(normalize=True)
    })

round(cross_tab,2)

,count,percentage
0,549,0.62
1,342,0.38


**Let's add a feature continuous feature**

In [7]:
model3 = smf.logit(formula='survived ~ fare', data=titanic).fit()
model3.params

Optimization terminated successfully.
         Current function value: 0.627143
         Iterations 6


Intercept   -0.941330
fare         0.015197
dtype: float64

In [9]:
log_odd=  -0.941330
odds = math.exp(log_odd)
print("Odds:", odds)
probability = odds/(1+odds)
print("Probability:", probability)

Odds: 0.39010864567516657
Probability: 0.28063176708443055


**Let's add a catagorical feature**

In [10]:
model2 = smf.logit(formula='survived ~ C(pclass)', data=titanic).fit()
model2.params

Optimization terminated successfully.
         Current function value: 0.607805
         Iterations 5


Intercept         0.530628
C(pclass)[T.2]   -0.639431
C(pclass)[T.3]   -1.670399
dtype: float64

In [13]:
log_odd = -0.63
odds = math.exp(log_odd)

probability = odds/(1+odds)

print(odds)
print(1/1.87)

0.5325918010068972
0.53475935828877


**Let's now combine features**

In [14]:
model2 = smf.logit(formula='survived ~ fare + C(sex) + age', data=titanic).fit()
model2.params

Optimization terminated successfully.
         Current function value: 0.501450
         Iterations 6


Intercept         0.934841
C(sex)[T.male]   -2.347599
fare              0.012773
age              -0.010570
dtype: float64

# Evaluate performance

In [18]:
model4 = smf.logit(formula='survived ~ fare + C(sex) + age', data=titanic).fit()
model4.summary()

Optimization terminated successfully.
         Current function value: 0.501450
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      710
Method:                           MLE   Df Model:                            3
Date:                Sat, 07 May 2022   Pseudo R-squ.:                  0.2576
Time:                        10:36:54   Log-Likelihood:                -358.04
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 1.419e-53
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.9348      0.239      3.910      0.000       0.466       1.403
C(sex)[T.male]    -2.3476      0.190    -12.359      0.000      -2.720      -1.975
fare               0.0128      0.003      4.738      0.000       0.007       0.018
age               -0.0106      0.006     -1.627      0.104      -0.023       0.002
==================================================================================
"""

Key things to note: 
- Model
- P Value, Z score, 95% cofidence interval
- Pseudo R-squ
- Log likelihood

In [6]:
# model4.summary()

# Multicolinearity issues in Linear/Logistic Regression

In [19]:
mpg = sns.load_dataset('mpg').dropna().drop(columns=['origin', 'name', 'displacement'])
mpg.corr().style.background_gradient(cmap='coolwarm')

,mpg,cylinders,horsepower,weight,acceleration,model_year
mpg,1.000000,-0.777618,-0.778427,-0.832244,0.423329,0.580541
cylinders,-0.777618,1.000000,0.842983,0.897527,-0.504683,-0.345647
horsepower,-0.778427,0.842983,1.000000,0.864538,-0.689196,-0.416361
weight,-0.832244,0.897527,0.864538,1.000000,-0.416839,-0.309120
acceleration,0.423329,-0.504683,-0.689196,-0.416839,1.000000,0.290316
model_year,0.580541,-0.345647,-0.416361,-0.309120,0.290316,1.000000


In [20]:
smf.ols(formula='weight ~ cylinders + horsepower', data=mpg).fit().params

Intercept     528.876711
cylinders     290.356425
horsepower      8.231070
dtype: float64

In [21]:
mpg['lin_comb'] = 10 * mpg['cylinders'] - 0.3 * mpg['horsepower']
mpg.head(3)

,mpg,cylinders,horsepower,weight,acceleration,model_year,lin_comb
0,18.0,8,130.0,3504,12.0,70,41.0
1,15.0,8,165.0,3693,11.5,70,30.5
2,18.0,8,150.0,3436,11.0,70,35.0


In [22]:
smf.ols(formula='weight ~ cylinders + horsepower + lin_comb', data=mpg).fit().params

Intercept     528.876711
cylinders       3.375029
horsepower     16.840512
lin_comb       28.698140
dtype: float64

In [25]:
# # Now, change just a bit one single observation by 1% just on one feature
mpg.loc[0,'horsepower'] = mpg.loc[0,'horsepower']*1.01
smf.ols(formula='weight ~ cylinders + horsepower + lin_comb', data=mpg).fit().params

Intercept      524.838981
cylinders     5954.582173
horsepower    -161.691947
lin_comb      -566.220542
dtype: float64

In [27]:
# Statsmodels gives us a clear WARNING [2]
# Summary table also reads 'Covariance Type: nonrobust'
smf.ols(formula='weight ~ cylinders + horsepower + lin_comb', data=mpg).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 weight   R-squared:                       0.846
Model:                            OLS   Adj. R-squared:                  0.845
Method:                 Least Squares   F-statistic:                     712.9
Date:                Sat, 07 May 2022   Prob (F-statistic):          1.99e-157
Time:                        10:45:54   Log-Likelihood:                -2832.3
No. Observations:                 392   AIC:                             5673.
Df Residuals:                     388   BIC:                             5689.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    524.8390     56.865      9.230      0.000     413.038     636.640
cylinders   5954.5822   4286.694      1.389      0.166   -2473.473    1.44e+04
horsepower  -161.6919    128.597     -1.257      0.209    -414.526      91.142
lin_comb    -566.2205    428.515     -1.321      0.187   -1408.722     276.281
==============================================================================
Omnibus:                       12.222   Durbin-Watson:                   1.199
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               24.236
Skew:                           0.069   Prob(JB):                     5.46e-06
Kurtosis:                       4.210   Cond. No.                     2.91e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.91e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
# mpg['lin_comb'] = mpg['lin_comb'] +  0.05 * np.random.rand(mpg.shape[0])
# np.linalg.matrix_rank(mpg)

In [14]:
# smf.ols(formula='weight ~ cylinders + horsepower + lin_comb', data=mpg).fit().params

In [15]:
# # Again, change just a bit one single observation in the dataset and check the OLS results
# mpg.loc[0,'horsepower'] = mpg.loc[0,'horsepower']*0.8
# smf.ols(formula='weight ~ cylinders + horsepower + lin_comb', data=mpg).fit().params

## How to detect multicolinearity

In [28]:
# Correlation matix is not sufficient to detect soft or event strict multicolinearity
mpg.corr().style.background_gradient(cmap='coolwarm')

,mpg,cylinders,horsepower,weight,acceleration,model_year,lin_comb
mpg,1.000000,-0.777618,-0.778454,-0.832244,0.423329,0.580541,-0.445237
cylinders,-0.777618,1.000000,0.843139,0.897527,-0.504683,-0.345647,0.762622
horsepower,-0.778454,0.843139,1.000000,0.864541,-0.689335,-0.416593,0.295197
weight,-0.832244,0.897527,0.864541,1.000000,-0.416839,-0.309120,0.554685
acceleration,0.423329,-0.504683,-0.689335,-0.416839,1.000000,0.290316,-0.067730
model_year,0.580541,-0.345647,-0.416593,-0.309120,0.290316,1.000000,-0.113309
lin_comb,-0.445237,0.762622,0.295197,0.554685,-0.067730,-0.113309,1.000000


In [29]:
mpg_scaled = mpg.copy()

for feature in mpg_scaled.columns:
    mu = mpg[feature].mean()
    sigma = mpg[feature].std()
    mpg_scaled[feature] = mpg_scaled[feature].apply(lambda x: (x-mu)/sigma)
    
mpg_scaled

,mpg,cylinders,horsepower,weight,acceleration,model_year,lin_comb
0,-0.697747,1.482053,0.730909,0.619748,-1.283618,-1.623241,1.834954
1,-1.082115,1.482053,1.572221,0.842258,-1.464852,-1.623241,0.741568
2,-0.697747,1.482053,1.182569,0.539692,-1.646086,-1.623241,1.210162
3,-0.953992,1.482053,1.182569,0.536160,-1.283618,-1.623241,1.210162
4,-0.825870,1.482053,0.922800,0.554997,-1.827320,-1.623241,1.522558
...,...,...,...,...,...,...,...
393,0.455359,-0.862911,-0.479950,-0.220842,0.021267,1.634321,-0.955783
394,2.633448,-0.862911,-1.363162,-0.997859,3.283479,1.634321,0.106363
395,1.095974,-0.862911,-0.531903,-0.803605,-1.428605,1.634321,-0.893304
396,0.583482,-0.862911,-0.661788,-0.415097,1.108671,1.634321,-0.737106


In [31]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
# compute VIF factor for feature index 0
vif(mpg_scaled.values, 0)

5.23757224818339

In [32]:
df = pd.DataFrame()

df["features"] = mpg_scaled.columns

df["vif_index"] = [vif(mpg_scaled.values, i) for i in range(mpg_scaled.shape[1])]

round(df.sort_values(by="vif_index", ascending = False),2)

,features,vif_index
1,cylinders,189828.71
2,horsepower,87032.48
6,lin_comb,60101.90
3,weight,11.25
0,mpg,5.24
4,acceleration,2.61
5,model_year,1.91
